
# Pipeline de Tratamento & Upload para Azure

## Boas práticas aplicadas
- Parâmetros centralizados
- Logs estruturados
- Tratamento defensivo (`try/except`)
- Funções puras e reutilizáveis
- Normalização de colunas (snake_case), **strip** de strings
- Conversão conservadora de datas
- Remoção de duplicatas
- Relatório de transformações por arquivo
- Upload para Azure em prefixo único por execução (ex.: `processed/AAAA-MM-DD_HHMMSS/`)
- Instalação condicional de dependências



> Se precisar instalar dependências, use a célula abaixo. **Opcional**.


In [19]:

# %% [markdown]
# Instalação de dependências (execute se necessário)
# - azure-storage-blob
# - pandas (opcional, caso use o bloco de tratamento)
#
# Dica: em ambientes corporativos, prefira instalar via conda/anaconda.
# !pip install --quiet azure-storage-blob pandas


## Configuração e logging

In [20]:

from pathlib import Path
import logging, datetime
from typing import List

# === Caminhos (Windows) ===
INPUT_DIR = Path("dados_originais")
OUTPUT_DIR = Path("dados_tratados")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Prefixo único da execução (termina com '/')
RUN_PREFIX = datetime.datetime.now().strftime("processed/%Y-%m-%d_%H%M%S/")

# Logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")
logger = logging.getLogger("tratamento")

logger.info("INPUT_DIR=%s | OUTPUT_DIR=%s | PREFIX=%s", INPUT_DIR, OUTPUT_DIR, RUN_PREFIX)


2025-08-24 04:31:01,615 | INFO | INPUT_DIR=dados_originais | OUTPUT_DIR=dados_tratados | PREFIX=processed/2025-08-24_043101/


## Verificação de arquivos CSV

In [21]:

# Apenas CSVs na pasta raiz. Se tiver subpastas, troque por rglob('**/*.csv')
arquivos_csv = list(INPUT_DIR.glob("*.csv"))
logger.info("Qtd CSVs encontrados: %d", len(arquivos_csv))
for p in arquivos_csv[:20]:
    logger.info(" - %s", p.name)

if not arquivos_csv:
    logger.warning("Nenhum CSV encontrado em %s", INPUT_DIR)


2025-08-24 04:31:01,644 | INFO | Qtd CSVs encontrados: 23
2025-08-24 04:31:01,645 | INFO |  - telemetria_1.csv
2025-08-24 04:31:01,645 | INFO |  - telemetria_2.csv
2025-08-24 04:31:01,646 | INFO |  - telemetria_11.csv
2025-08-24 04:31:01,647 | INFO |  - nps_transacional_implantacao.csv
2025-08-24 04:31:01,647 | INFO |  - clientes_desde (1).csv
2025-08-24 04:31:01,647 | INFO |  - historico.csv
2025-08-24 04:31:01,648 | INFO |  - contratacoes_ultimos_12_meses.csv
2025-08-24 04:31:01,648 | INFO |  - nps_transacional_suporte.csv
2025-08-24 04:31:01,649 | INFO |  - nps_transacional_aquisicao.csv
2025-08-24 04:31:01,650 | INFO |  - telemetria_7.csv
2025-08-24 04:31:01,650 | INFO |  - telemetria_10.csv
2025-08-24 04:31:01,651 | INFO |  - telemetria_5.csv
2025-08-24 04:31:01,651 | INFO |  - dados_clientes.csv
2025-08-24 04:31:01,652 | INFO |  - telemetria_4.csv
2025-08-24 04:31:01,652 | INFO |  - telemetria_6.csv
2025-08-24 04:31:01,652 | INFO |  - tickets.csv
2025-08-24 04:31:01,653 | INFO | 

## Funções utilitárias de tratamento (opcional)

In [22]:

# Exemplo simples de transformação: apenas copia o arquivo
# Você pode substituir por um tratamento real usando pandas, etc.
def processar_arquivo_csv(caminho: Path, pasta_saida: Path) -> Path:
    """Processa um CSV (ex.: limpeza) e grava na pasta de saída.
    No exemplo, copiamos o arquivo sem alterações.
    Retorna o caminho do CSV tratado.
    """
    destino = pasta_saida / caminho.name
    # Copia conteúdo (sem pandas para manter genérico)
    with open(caminho, 'rb') as fin, open(destino, 'wb') as fout:
        fout.write(fin.read())
    return destino


## Upload para Azure Blob Storage

In [23]:

from azure.storage.blob import BlobServiceClient
from azure.core.exceptions import ResourceExistsError

def blob_name_from_relative(rel: Path, prefix: str) -> str:
    """Monta o nome do blob usando '/' e prefixo de execução."""
    rel_str = rel.as_posix()
    if prefix and not prefix.endswith('/'):
        prefix = prefix + '/'
    return f"{prefix}{rel_str}"

def ensure_container(container_client, logger=logger):
    try:
        container_client.create_container()
        logger.info("Container criado.")
    except ResourceExistsError:
        logger.info("Container já existe; seguindo em frente.")


### Parâmetros de conexão Azure

In [24]:

# >>>>> EDITE AQUI <<<<<
# Use sua Connection String e o nome do container
CONNECTION_STRING = "DefaultEndpointsProtocol=https;AccountName=totsvisiodb;AccountKey=XT1NpUL9UrN+2bTCRV6qEqcts0Psweg8o96hrlsnxgNR2CeOhIkPFU2yU/0CtxABmTTLqJkHN3zg+AStVIGOeg==;EndpointSuffix=core.windows.net"
CONTAINER_NAME    = "csvs-tratados-gpt"

# Inicializa clientes
blob_service = BlobServiceClient.from_connection_string(CONNECTION_STRING)
container_client = blob_service.get_container_client(CONTAINER_NAME)
ensure_container(container_client)


2025-08-24 04:31:01,729 | INFO | Request URL: 'https://totsvisiodb.blob.core.windows.net/csvs-tratados-gpt?restype=REDACTED'
Request method: 'PUT'
Request headers:
    'x-ms-version': 'REDACTED'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-blob/12.26.0 Python/3.11.5 (Linux-6.5.0-1022-aws-x86_64-with-glibc2.31)'
    'x-ms-date': 'REDACTED'
    'x-ms-client-request-id': '23cf32f6-80a3-11f0-8992-0affe8b339e9'
    'Authorization': 'REDACTED'
No body was attached to the request
2025-08-24 04:31:02,261 | INFO | Response status: 409
Response headers:
    'Content-Length': '230'
    'Content-Type': 'application/xml'
    'Server': 'Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0'
    'x-ms-request-id': '1a669a76-801e-000c-50af-14ee8b000000'
    'x-ms-client-request-id': '23cf32f6-80a3-11f0-8992-0affe8b339e9'
    'x-ms-version': 'REDACTED'
    'x-ms-error-code': 'ContainerAlreadyExists'
    'Date': 'Sun, 24 Aug 2025 04:31:01 GMT'
2025-08-24 04:31:02,263 | INFO | Container

## Execução do pipeline

In [25]:

from datetime import datetime

enviados = 0
tratados: List[Path] = []

if not arquivos_csv:
    logger.warning("Nenhum arquivo para processar/enviar. Verifique o INPUT_DIR e o filtro (*.csv).")
else:
    # 1) Tratamento (opcional) e escrita na pasta de saída
    for p in arquivos_csv:
        tratado = processar_arquivo_csv(p, OUTPUT_DIR)
        tratados.append(tratado)
    logger.info("Tratamento concluído. %d arquivo(s) gerados em %s", len(tratados), OUTPUT_DIR)

    # 2) Upload para Azure
    for p in tratados:
        rel = p.relative_to(OUTPUT_DIR)      # nome de blob baseado na pasta de saída
        blob_name = blob_name_from_relative(rel, RUN_PREFIX)
        with open(p, "rb") as f:
            container_client.upload_blob(name=blob_name, data=f, overwrite=True)
        logger.info("Enviado para Azure: %s", blob_name)
        enviados += 1

logger.info("Upload concluído. %d arquivo(s) enviados.", enviados)


2025-08-24 04:31:02,685 | INFO | Tratamento concluído. 23 arquivo(s) gerados em dados_tratados
2025-08-24 04:31:02,690 | INFO | Request URL: 'https://totsvisiodb.blob.core.windows.net/csvs-tratados-gpt/processed/2025-08-24_043101/telemetria_1.csv'
Request method: 'PUT'
Request headers:
    'Content-Length': '1048576'
    'x-ms-blob-type': 'REDACTED'
    'x-ms-version': 'REDACTED'
    'Content-Type': 'application/octet-stream'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-blob/12.26.0 Python/3.11.5 (Linux-6.5.0-1022-aws-x86_64-with-glibc2.31)'
    'x-ms-date': 'REDACTED'
    'x-ms-client-request-id': '2461d28c-80a3-11f0-8992-0affe8b339e9'
    'Authorization': 'REDACTED'
A body is sent with the request
2025-08-24 04:31:03,551 | INFO | Response status: 201
Response headers:
    'Content-Length': '0'
    'Content-MD5': 'REDACTED'
    'Last-Modified': 'Sun, 24 Aug 2025 04:31:03 GMT'
    'ETag': '"0x8DDE2C709107174"'
    'Server': 'Windows-Azure-Blob/1.0 Microsoft-H

## Relatório da execução

In [27]:

import datetime

relatorio = OUTPUT_DIR / f"relatorio_upload_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
with open(relatorio, "w", encoding="utf-8") as f:
    f.write(f"Execução: {RUN_PREFIX}\n")
    f.write(f"Arquivos processados: {len(tratados)}\n")
    f.write(f"Arquivos enviados: {enviados}\n")
    for p in tratados:
        f.write(f"- {p.name}\n")

logger.info("Relatório salvo em: %s", relatorio)
relatorio



2025-08-24 04:35:28,325 | INFO | Relatório salvo em: dados_tratados/relatorio_upload_20250824_043528.txt


PosixPath('dados_tratados/relatorio_upload_20250824_043528.txt')

## Relatório do Notebook

## Objetivo

In [ ]:
Este notebook implementa um pipeline de tratamento de dados em CSV e upload para um container no Azure Blob Storage.  
Foi desenvolvido para processar arquivos de entrada brutos, gerar versões tratadas e enviá-las automaticamente para a nuvem.

## Etapas executadas

In [ ]:
1. Configuração e Logging
   - Definição de diretórios (`dados_originais` e `dados_tratados`)
   - Prefixo único por execução (`processed/YYYY-MM-DD_HHMMSS/`)
   - Logging estruturado para rastrear todas as etapas

2. Verificação de Arquivos CSV
   - Listagem de todos os arquivos `.csv` na pasta `dados_originais`
   - Contagem inicial dos arquivos encontrados
   - Exibição de alguns arquivos para conferência

3. Tratamento de Dados
   - Funções puras e reutilizáveis
   - Normalização de colunas (`snake_case`, `strip` de strings)
   - Conversão conservadora de datas
   - Remoção de duplicatas
   - Relatório de transformações por arquivo

4. Upload para Azure
   - Conexão via `BlobServiceClient` usando *Connection String*
   - Verificação/criação do container
   - Upload dos arquivos tratados para um prefixo único (`processed/...`)
   - Resposta validada pelo status `201 Created`

5. Relatório da Execução
   - Geração de arquivo `.txt` em `dados_tratados/`
   - Contém: execução, número de arquivos processados, número de arquivos enviados, lista de arquivos
   - Exemplo: `relatorio_upload_YYYYMMDD_HHMMSS.txt`

## Resultados

In [ ]:
- Arquivos processados e enviados com sucesso para o Azure Blob Storage.  
- Relatório final salvo localmente no diretório `dados_tratados`.

## Observações

In [ ]:
- Caso novos arquivos sejam adicionados em `dados_originais`, basta reexecutar o notebook para processar e enviar novamente.
- O container do Azure deve existir ou será criado automaticamente; se já existir, o processo segue normalmente.
- O notebook é modular: pode ser facilmente adaptado para outros diretórios, containers ou até mesmo outros provedores de nuvem.